# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Загрузка данных и изучение общей информации

In [37]:
import pandas as pd
from IPython.display import display
from pymystem3 import Mystem
m = Mystem() 
from collections import Counter

In [38]:
data = pd.read_csv('/Users/vintera/Git/projects/my_projects/dataset/project_01/data.csv')

In [39]:
data.info()
data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [40]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [41]:
data.tail()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


In [42]:
data.sample(frac=0.1, random_state=1)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
1383,0,353802.811675,37,среднее,1,вдовец / вдова,2,F,пенсионер,0,216452.226085,строительство недвижимости
300,1,-359.193975,33,СРЕДНЕЕ,1,гражданский брак,1,M,сотрудник,0,223001.623994,на проведение свадьбы
6565,2,-1064.854333,35,среднее,1,гражданский брак,1,F,компаньон,0,163591.209323,свадьба
17027,0,NaN,48,высшее,0,гражданский брак,1,F,сотрудник,0,NaN,операции с жильем
4077,0,-7059.100220,45,высшее,0,гражданский брак,1,F,компаньон,1,194820.185757,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
3871,0,-3161.018082,30,среднее,1,женат / замужем,0,F,сотрудник,0,195225.109850,высшее образование
12523,0,-311.045592,45,среднее,1,женат / замужем,0,M,сотрудник,0,173858.235033,жилье
7530,2,-3425.051465,49,высшее,0,женат / замужем,0,M,компаньон,0,147165.593376,операции с недвижимостью
17091,0,-3757.852014,44,среднее,1,гражданский брак,1,F,компаньон,0,102418.840829,жилье


In [43]:
data.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

#### Комментарий
С задаными названиями столбцов работать удобно. Исправления здесь излишни.

In [44]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

#### Комментарий
20 детей? Теоретически возможно.. А вот отрицательные значения следует устранить.

In [45]:
data.loc[data['days_employed'] >= 0].sort_values(by='days_employed')

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
20444,0,328728.720605,72,среднее,1,вдовец / вдова,2,F,пенсионер,0,96519.339647,покупка жилья для семьи
9328,2,328734.923996,41,высшее,0,женат / замужем,0,M,пенсионер,0,126997.497760,операции со своей недвижимостью
17782,0,328771.341387,56,среднее,1,женат / замужем,0,F,пенсионер,0,68648.047062,операции с коммерческой недвижимостью
14783,0,328795.726728,62,высшее,0,женат / замужем,0,F,пенсионер,0,79940.196752,на покупку своего автомобиля
7229,1,328827.345667,32,среднее,1,гражданский брак,1,F,пенсионер,0,122162.965695,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
7794,0,401663.850046,61,среднее,1,гражданский брак,1,F,пенсионер,0,48286.441362,свадьба
2156,0,401674.466633,60,среднее,1,женат / замужем,0,M,пенсионер,0,325395.724541,автомобили
7664,1,401675.093434,61,среднее,1,женат / замужем,0,F,пенсионер,0,126214.519212,операции с жильем
10006,0,401715.811749,69,высшее,0,Не женат / не замужем,4,F,пенсионер,0,57390.256908,получение образования


#### Комментарий
Мы уже знаем, что в трудовом стаже имеются пропуски и отрицательные значения. С положительными значениеями не лучше - самый непродолжительный стаж здесь 923 года, если проводить пересчет в днях. Если предположить, что система сбора данных дала ошибку и считала стаж в часах, то стаж 328827/24/356 = 38 лет для 32-летнего заемщика тоже кажется сомнительным.

In [46]:
data.loc[data['dob_years'] < 18].sort_values(by='dob_years')

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,346541.618895,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
13968,1,-1018.525283,0,среднее,1,женат / замужем,0,F,сотрудник,1,155341.706429,свой автомобиль
13741,0,NaN,0,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,на проведение свадьбы
13521,0,-681.907359,0,высшее,0,Не женат / не замужем,4,M,сотрудник,0,115165.323707,строительство жилой недвижимости
13439,0,-1036.644001,0,среднее,1,женат / замужем,0,M,сотрудник,1,271371.522623,операции с жильем
...,...,...,...,...,...,...,...,...,...,...,...,...
6778,0,-1478.092467,0,высшее,0,Не женат / не замужем,4,F,сотрудник,0,157362.970952,получение высшего образования
6670,0,NaN,0,Высшее,0,в разводе,3,F,пенсионер,0,NaN,покупка жилой недвижимости
6411,0,NaN,0,высшее,0,гражданский брак,1,F,пенсионер,0,NaN,свадьба
7344,0,-401.461262,0,среднее,1,женат / замужем,0,M,сотрудник,0,158913.767700,операции с жильем


#### Комментарий
101 заемщик с возрастом 0 лет.

In [47]:
data['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

#### Комментарий
5 видов образования отображаются как 15 из-за различия в регистрах.

In [48]:
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

#### Комментарий
Пол XNA? Учитывая, что такое значение единичено, на результаты исследования оно не повлияет.

### Вывод
Предоставленная таблица состоит из 21525 строк и 12 столбцов, содержащих различные типы данных. В столбцах "days_employed" и "total_income" заполнены 19351 ячеек, то есть данные в 2174 ячейках (10%) этих столбцов отсутствуют. Образовавшиеся пропущенные значения возможно были вызваны ошибками в системе сбора данных.
Также, при подготовке данных для работы с ними, следует обратить внимание на отрицательные значения и чрезмерно высокие показатели в столбце "days_employed", возраст 0 лет у заемщиков и привести к единому регистру данные об образовании.

## Предобработка и исследование данных

### Обработка пропусков

In [49]:
# Находим медианные значения доходов для различных видов занятости.
income_by_income_type = data.groupby('income_type').agg({'total_income' : ['median']})
income_by_income_type.columns = ['median_total_income']
data = data.merge(income_by_income_type, on= ['income_type'])
data[['income_type', 'total_income', 'median_total_income']][data['total_income'].isna()]

,income_type,total_income,median_total_income
46,сотрудник,NaN,142594.396847
47,сотрудник,NaN,142594.396847
49,сотрудник,NaN,142594.396847
54,сотрудник,NaN,142594.396847
55,сотрудник,NaN,142594.396847
...,...,...,...
21457,госслужащий,NaN,150447.935283
21489,госслужащий,NaN,150447.935283
21511,госслужащий,NaN,150447.935283
21513,госслужащий,NaN,150447.935283


In [50]:
# Заменяем пропущенные значения в доходах, на медианные, с учетом вида занятости.
data.loc[data['total_income'].isna(), 'total_income'] = data.loc[data['total_income'].isna(), 'median_total_income']

In [51]:
# Находим медианное значение в столбце стажа, среди значений ниже 0, как среди самых адекватных.
median_days_employed = data.loc[data['days_employed'] < 0, 'days_employed'].median()
median_days_employed

-1630.0193809778218

In [52]:
# Заменяем пропуски и очень большие значения в столбце со стажем на медианные.
data['days_employed'] = data['days_employed'].fillna(median_days_employed)
data.loc[data['days_employed'] > 0, 'days_employed'] = median_days_employed

#### Комментарий
Учитывая, что бОльшая часть данных о трудовом стаже имела отрицательные показатели, а их натуральные значения поддавались логике при пересчете на годы, можно предположить, что при заполнении данных, отсчет стажа шел в обратном порядке. 
Также помним, что строк с отрицательным значением (-1) в данных о детях ничтожно мало (47).
Обращаем отрицательные значения в положительные, возвращая модуль числа.

In [53]:
# Обращаем отрицательные значения в положительные, возвращая модуль числа.
data['days_employed'] = data['days_employed'].abs()
data['children'] = data['children'].abs()
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21525 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   children             21525 non-null  int64  
 1   days_employed        21525 non-null  float64
 2   dob_years            21525 non-null  int64  
 3   education            21525 non-null  object 
 4   education_id         21525 non-null  int64  
 5   family_status        21525 non-null  object 
 6   family_status_id     21525 non-null  int64  
 7   gender               21525 non-null  object 
 8   income_type          21525 non-null  object 
 9   debt                 21525 non-null  int64  
 10  total_income         21525 non-null  float64
 11  purpose              21525 non-null  object 
 12  median_total_income  21525 non-null  float64
dtypes: float64(3), int64(5), object(5)
memory usage: 2.3+ MB


In [54]:
# Избавляемся от заемщиков с возрастом ниже допустимого и сбрасываем индексы.
data = data[data['dob_years'] >= 18].reset_index(drop=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21424 entries, 0 to 21423
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   children             21424 non-null  int64  
 1   days_employed        21424 non-null  float64
 2   dob_years            21424 non-null  int64  
 3   education            21424 non-null  object 
 4   education_id         21424 non-null  int64  
 5   family_status        21424 non-null  object 
 6   family_status_id     21424 non-null  int64  
 7   gender               21424 non-null  object 
 8   income_type          21424 non-null  object 
 9   debt                 21424 non-null  int64  
 10  total_income         21424 non-null  float64
 11  purpose              21424 non-null  object 
 12  median_total_income  21424 non-null  float64
dtypes: float64(3), int64(5), object(5)
memory usage: 2.1+ MB


In [55]:
# Переписываем данные об образовании в нижнем регистре.
data['education'] = data['education'].str.lower()
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [56]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21424 entries, 0 to 21423
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   children             21424 non-null  int64  
 1   days_employed        21424 non-null  float64
 2   dob_years            21424 non-null  int64  
 3   education            21424 non-null  object 
 4   education_id         21424 non-null  int64  
 5   family_status        21424 non-null  object 
 6   family_status_id     21424 non-null  int64  
 7   gender               21424 non-null  object 
 8   income_type          21424 non-null  object 
 9   debt                 21424 non-null  int64  
 10  total_income         21424 non-null  float64
 11  purpose              21424 non-null  object 
 12  median_total_income  21424 non-null  float64
dtypes: float64(3), int64(5), object(5)
memory usage: 2.1+ MB


#### Вывод

После исправления ошибок, выявленных в первой главе, мы получаем таблицу состоящую из 21424 строк, что приблизительно на 0,5% меньше изначального количества. Ценой утраченных строк мы получаем таблицу без пропусков, аномальных показателей и значений равных 0, с данными из которой уже можно работать.

### Замена типа данных

In [57]:
data['days_employed'] = data['days_employed'].astype('int')

In [58]:
data['total_income'] = data['total_income'].astype('int')

In [59]:
data['median_total_income'] = data['median_total_income'].astype('int')

In [60]:
data.dtypes

children                int64
days_employed           int64
dob_years               int64
education              object
education_id            int64
family_status          object
family_status_id        int64
gender                 object
income_type            object
debt                    int64
total_income            int64
purpose                object
median_total_income     int64
dtype: object

#### Вывод

Для удобства пользования таблицей данные в столбцах "days_employed" и "total_income" были заменены на целые числа.

### Обработка дубликатов

In [61]:
data.duplicated().sum()

71

In [62]:
data = data.drop_duplicates(inplace=False).reset_index(drop=True)

#### Вывод

Учитывая то, что в предоставленных данных отсутствуют уникальные идентификаторы, поиск дубликатов производился на предмет полного совпадения. Так как вреоятность полного совпадения данных разных заемщиков по всем параметрам крайне низка, можно предположить, что природой их появления является сбой в системе сбора данных. Удаляем повторяющиеся 71 строку.

### Лемматизация

In [63]:
# Для наглядности.
data['purpose'].value_counts()

свадьба                                   786
на проведение свадьбы                     764
сыграть свадьбу                           760
операции с недвижимостью                  672
покупка коммерческой недвижимости         658
покупка жилья для сдачи                   649
операции с коммерческой недвижимостью     648
операции с жильем                         646
жилье                                     640
покупка жилья                             640
покупка жилья для семьи                   637
строительство собственной недвижимости    633
недвижимость                              629
операции со своей недвижимостью           627
строительство жилой недвижимости          621
покупка своего жилья                      619
строительство недвижимости                619
покупка недвижимости                      618
ремонт жилью                              605
покупка жилой недвижимости                603
на покупку своего автомобиля              502
заняться высшим образованием      

In [64]:
# Получаем информацию о словах и количестве их употреблений в столбце "Цель".
purpose_list = data['purpose'].unique()
lemmas = m.lemmatize(' '.join(purpose_list))
Counter(lemmas)

Counter({'покупка': 10,
         ' ': 96,
         'жилье': 7,
         'приобретение': 1,
         'автомобиль': 9,
         'дополнительный': 2,
         'образование': 9,
         'на': 4,
         'проведение': 1,
         'свадьба': 3,
         'для': 2,
         'семья': 1,
         'коммерческий': 2,
         'недвижимость': 10,
         'строительство': 3,
         'собственный': 1,
         'свой': 4,
         'подержать': 2,
         'сыграть': 1,
         'операция': 4,
         'со': 1,
         'получение': 3,
         'заниматься': 2,
         'с': 5,
         'сделка': 2,
         'профильный': 1,
         'высокий': 3,
         'жилой': 2,
         'сдача': 1,
         'ремонт': 1,
         '\n': 1})

#### Вывод

Проанализировав полученную информацию, можем выделить основные группы целей получения кредита: "Недвижимость", "Автотранспорт", "Образование" и, как ни странно, "Свадьба".

### Категоризация данных

In [65]:
# Категоризируем данные о целях займа по четырем основным группам, используя итоги лемматизации. 
def purpose_grouping(row_values):
    lemm = m.lemmatize(row_values['purpose'])
    if 'автомобиль' in lemm:
        return 'автокредит'
    
    if 'образование' in lemm:
        return 'кредит на образование'
    
    if 'свадьба' in lemm:
        return 'кредит на свадьбу'
    
    if 'жилье' or 'недвижимость' in lemm:
        return 'операции с недвижимостью'
    
    return 'иные цели'

data['purpose_group'] = data.apply(purpose_grouping, axis=1)
data['purpose_group'].value_counts()

операции с недвижимостью    10764
автокредит                   4284
кредит на образование        3995
кредит на свадьбу            2310
Name: purpose_group, dtype: int64

In [66]:
# Столбец содержащий информацию о доходах заёмщиков, для удобства постановки выводов, разобьем на 4 категории, которые приблизительно будут равны по своим значениям.
def total_income_grouping(row):
    income = row['total_income']
    
    if income <= 110000:
        return 'до 110 тысяч'
    
    if 110000 < income <= 145000:
        return 'от 110 до 145 тысяч'
    
    if 145000 < income <= 200000:
        return 'от 145 до 200 тысяч'
    
    return 'свыше 200 тысяч'

data['income_group'] = data.apply(total_income_grouping, axis=1)
data['income_group'].value_counts()

до 110 тысяч           5611
от 110 до 145 тысяч    5466
от 145 до 200 тысяч    5235
свыше 200 тысяч        5041
Name: income_group, dtype: int64

In [67]:
# Информацию о количестве детей заёмщиков разобьем на три группы.
def children_amount_grouping(row):
    children = row['children']
    
    if children == 0:
        return 'бездетные'
    
    if children <= 2:
        return '1 - 2 ребенка'
    
    return 'многодетные'

data['children_group'] = data.apply(children_amount_grouping, axis=1)
data['children_group'].value_counts()

бездетные        14022
1 - 2 ребенка     6878
многодетные        453
Name: children_group, dtype: int64

#### Вывод

Для удобства подготовки ответов на выдвинутые заказчиком вопросы, в данном разделе мы категоризировали интересующие данные о заёмщиках.

## Исследовательский анализ данных (EDA)

### Зависимость между наличием детей и возвратом кредита в срок

In [68]:
# Таблица с интересующими данными.
required_data = data[['children_group', 'family_status', 'income_group', 'purpose_group', 'debt']]
required_data

,children_group,family_status,income_group,purpose_group,debt
0,1 - 2 ребенка,женат / замужем,свыше 200 тысяч,операции с недвижимостью,0
1,1 - 2 ребенка,женат / замужем,от 110 до 145 тысяч,автокредит,0
2,бездетные,женат / замужем,от 145 до 200 тысяч,операции с недвижимостью,0
3,многодетные,женат / замужем,свыше 200 тысяч,кредит на образование,0
4,бездетные,женат / замужем,от 110 до 145 тысяч,кредит на образование,0
...,...,...,...,...,...
21348,бездетные,гражданский брак,свыше 200 тысяч,операции с недвижимостью,0
21349,бездетные,женат / замужем,свыше 200 тысяч,операции с недвижимостью,0
21350,бездетные,гражданский брак,свыше 200 тысяч,кредит на свадьбу,0
21351,бездетные,Не женат / не замужем,до 110 тысяч,операции с недвижимостью,0


In [69]:
answer_1 = required_data.groupby('children_group').agg({'debt' : ['count', 'sum', 'mean']})
answer_1.columns = ['общее кол-во заёмщиков', 'из них, имевших долги', '%невозврата']
answer_1 = answer_1.sort_values(by='%невозврата', ascending=False)
answer_1 = answer_1.style.format({'%невозврата':'{:.2%}'})
answer_1

,общее кол-во заёмщиков,"из них, имевших долги",%невозврата
children_group,,,
1 - 2 ребенка,6878,636,9.25%
многодетные,453,39,8.61%
бездетные,14022,1058,7.55%


#### Вывод

Несмотря на то, что бездетные заемщики почти в два раза чаще обращаются за кредитом и впоследствии имеют по ним долги, заёмщики, имеющие 1-2 детей почти на 2% чаще не возвращают кредит в срок, по сравнению с бездетными.

### Зависимость между семейным положением и возвратом кредита в срок

In [70]:
answer_2 = required_data.groupby('family_status').agg({'debt' : ['count', 'sum', 'mean']})
answer_2.columns = ['общее кол-во заёмщиков', 'из них, имевших долги', '%невозврата']
answer_2 = answer_2.sort_values(by='%невозврата', ascending=False)
answer_2 = answer_2.style.format({'%невозврата':'{:.2%}'})
answer_2

,общее кол-во заёмщиков,"из них, имевших долги",%невозврата
family_status,,,
Не женат / не замужем,2794,273,9.77%
гражданский брак,4130,386,9.35%
женат / замужем,12290,927,7.54%
в разводе,1185,85,7.17%
вдовец / вдова,954,62,6.50%


#### Вывод

Самая многочисленная группа заёмщиков, состоящих в супружеском браке, значительно превалирует над всеми остальными группами, вместе взятыми, по количеству займов и долгов по ним. Однако, не смотря на данный факт, заёмщики, не связавшие себя узами брака или находящиеся в гражданском браке, почти на 2% чаще не возвращают долги по кредитам в устанволенный срок, чем женатые/замужние. Исходя из результатов исследования, овдовевшие заёмщики являются самыми пунктуальными плательщиками. 

### Зависимость между уровнем дохода и возвратом кредита в срок

In [71]:
answer_3 = required_data.groupby('income_group').agg({'debt' : ['count', 'sum', 'mean']})
answer_3.columns = ['общее кол-во заёмщиков', 'из них, имевших долги', '%невозврата']
answer_3 = answer_3.sort_values(by='%невозврата', ascending=False)
answer_3 = answer_3.style.format({'%невозврата':'{:.2%}'})
answer_3

,общее кол-во заёмщиков,"из них, имевших долги",%невозврата
income_group,,,
от 110 до 145 тысяч,5466,480,8.78%
от 145 до 200 тысяч,5235,443,8.46%
до 110 тысяч,5611,453,8.07%
свыше 200 тысяч,5041,357,7.08%


#### Вывод

Зависимость между уровнем дохода и возвратом кредита в срок прослеживается незначительная. Группы заёмщиков, зарабатывающих до 200 тысяч имеют практически одинаковое количество невозвратов, варьирующееся от 8,09% до 8,8%. Зарабатывающие свыше 200 тысяч являются самыми добросовестными плательщиками по долгам с процентом невозврата - 7,09.

### Зависимость между целью кредита и возвратом кредита в срок

In [72]:
answer_4 = required_data.groupby('purpose_group').agg({'debt' : ['count', 'sum', 'mean']})
answer_4.columns = ['общее кол-во заёмщиков', 'из них, имевших долги', '%невозврата']
answer_4 = answer_4.sort_values(by='%невозврата', ascending=False)
answer_4 = answer_4.style.format({'%невозврата':'{:.2%}'})
answer_4

,общее кол-во заёмщиков,"из них, имевших долги",%невозврата
purpose_group,,,
автокредит,4284,400,9.34%
кредит на образование,3995,370,9.26%
кредит на свадьбу,2310,184,7.97%
операции с недвижимостью,10764,779,7.24%


#### Вывод

Исследуемая группа заёмщиков, обратившихся за получением кредита на операции с недвижимостью практически равна количеству заёмщиков по всем оставшимся целевым группам, вместе взятым. При этом, данная группа имеет самый низкий процент невозврата долга - 7,25%, что можно сказать и о взявших кредит на свадьбу - 7,97%. Невозврат по автокредиту и кредиту на образование выше почти на 2пп и составляет 9,34% и 9,28% соответственно.

## Общий вывод и рекомендации бизнесу
Исследовав полученные данные о заёмщиках, можно сделать выводы о том, что:

1. Процент невозврата по всем интересующим группам относительно невысок и не превышает 10%
2. Самыми добросовестными плательщиками являются - заёмщики, имеющие доходы свыше 200 тысяч, овдовевшие, не имеющие детей и кредитующиеся для проведения операций с недвижимостью.
3. Чаще всего не возвращают долг в установленные сроки плательщики по автокредиту, не состоящие в официальном браке, с доходом от 110 до 145 тысяч, имющие 1-2 детей.
4. Корреляция между исследуемыми группами заёмщиков и процентом невовзрата ими долга, крайне не высока и варьируется на уровне 1-3пп.